In [59]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.prompts.prompt import PromptTemplate


In [60]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = OpenAI(temperature=0)

In [61]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=False)

In [62]:
db_chain.run("which is most popular song?")

' The most popular song is 2 Minutes To Midnight.'

In [63]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [64]:
db_chain.run("which is most popular song?")



> Entering new SQLDatabaseChain chain...
which is most popular song? 
SQLQuery: SELECT Name, COUNT(*) AS Popularity FROM Track JOIN PlaylistTrack ON Track.TrackId = PlaylistTrack.TrackId GROUP BY Name ORDER BY Popularity DESC LIMIT 5;
SQLResult: [('2 Minutes To Midnight', 13), ('Wrathchild', 12), ('The Trooper', 12), ('The Number Of The Beast', 12), ('Iron Maiden', 12)]
Answer: The most popular song is 2 Minutes To Midnight.
> Finished chain.


' The most popular song is 2 Minutes To Midnight.'

In [65]:
db_chain.run("What are some example tracks by composer Johann Sebastian Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by composer Johann Sebastian Bach? 
SQLQuery: SELECT Name, Composer FROM Track WHERE Composer LIKE '%Johann Sebastian Bach%' LIMIT 5;
SQLResult: [('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Johann Sebastian Bach'), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Johann Sebastian Bach'), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', 'Johann Sebastian Bach'), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata', 'Johann Sebastian Bach'), ('Concerto No.2 in F Major, BWV1047, I. Allegro', 'Johann Sebastian Bach')]
Answer: Some example tracks by composer Johann Sebastian Bach are 'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', 'Toccata and Fugue in D Minor, BWV 565: I. Toccata', and 'Concerto No.2 in F Major, BWV1047, I. Allegro'.

' Some example tracks by composer Johann Sebastian Bach are \'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace\', \'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria\', \'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude\', \'Toccata and Fugue in D Minor, BWV 565: I. Toccata\', and \'Concerto No.2 in F Major, BWV1047, I. Allegro\'.'

In [66]:
db_chain.run("What are some example tracks by Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by Bach? 
SQLQuery: SELECT Name, Composer FROM Track WHERE Composer LIKE '%Bach%' LIMIT 5;
SQLResult: [('American Woman', 'B. Cummings/G. Peterson/M.J. Kale/R. Bachman'), ('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Johann Sebastian Bach'), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Johann Sebastian Bach'), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', 'Johann Sebastian Bach'), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata', 'Johann Sebastian Bach')]
Answer: Some example tracks by Bach are 'American Woman', 'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', and 'Toccata and Fugue in D Minor, BWV 565: I. Toccata'.
> Finished chain.


' Some example tracks by Bach are \'American Woman\', \'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace\', \'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria\', \'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude\', and \'Toccata and Fugue in D Minor, BWV 565: I. Toccata\'.'

In [67]:
db_chain.run("How many records are there?")



> Entering new SQLDatabaseChain chain...
How many records are there? 
SQLQuery: SELECT COUNT(*) FROM InvoiceLine;
SQLResult: [(2240,)]
Answer: There are 2240 records in the InvoiceLine table.
> Finished chain.


' There are 2240 records in the InvoiceLine table.'

In [68]:
_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone mentions record, they really mean Album.

Question: {input}"""

PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [69]:
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=True)

In [70]:
db_chain.run("How many records are there?")



> Entering new SQLDatabaseChain chain...
How many records are there? 
SQLQuery: SELECT COUNT(*) FROM Album;
SQLResult: [(347,)]
Answer: There are 347 records.
> Finished chain.


' There are 347 records.'

In [71]:
db_chain.run("List the total sales per country. Which country's customers spent the most?")



> Entering new SQLDatabaseChain chain...
List the total sales per country. Which country's customers spent the most? 
SQLQuery: SELECT Country, SUM(Total) AS TotalSales FROM Customer c JOIN Invoice i ON c.CustomerId = i.CustomerId GROUP BY Country ORDER BY TotalSales DESC;
SQLResult: [('USA', 523.0600000000003), ('Canada', 303.9599999999999), ('France', 195.09999999999994), ('Brazil', 190.09999999999997), ('Germany', 156.48), ('United Kingdom', 112.85999999999999), ('Czech Republic', 90.24000000000001), ('Portugal', 77.23999999999998), ('India', 75.25999999999999), ('Chile', 46.62), ('Ireland', 45.62), ('Hungary', 45.62), ('Austria', 42.62), ('Finland', 41.620000000000005), ('Netherlands', 40.62), ('Norway', 39.62), ('Sweden', 38.620000000000005), ('Poland', 37.620000000000005), ('Italy', 37.620000000000005), ('Denmark', 37.620000000000005), ('Australia', 37.620000000000005), ('Argentina', 37.620000000000005), ('Spain', 37.62), ('Belgium', 37.62)]
Answer: USA customers spent the most

' USA customers spent the most, with a total of $523.06.'